<center><b>CSIE604284 • Analitika Media Sosial</b></center>
<center><b>Fakultas Ilmu Komputer, Universitas Indonesia</b></center>
<center><b>Self Paced Learning - Topic Modeling</b></center>

<b>Instruksi Pengerjaan</b>:
* Self learning ini juga termasuk tugas
* Tugas ini adalah tugas individu namun Anda diperbolehkan berdiskusi dengan teman dalam proses pengerjaan tugas. 
* Anda diberikan berkas TopicModelling.ipynb dan kumpulan data
* Tuliskan jawaban Anda pada berkas ini dan kumpulkan melalui SCeLE dengan format penamaan <b>TopicModelling_Nama_NPM.ipynb</b> sebelum <b>Senin, 21 Maret 2022 pukul 22:00.</b>

# <b>A. Topic Modeling</b><br>
    
<i>Topic modeling</i> adalah suatu teknik <i>unsupervised learning</i> untuk mengekstrak topik-topik yang ada pada suatu kumpulan dokumen teks. Satu dokumen dapat memiliki lebih dari satu topik. <i>Topic modeling</i> dapat membantu mengeksplorasi dokumen teks dengan jumlah yang besar untuk menemukan kelompok kata, kesamaan antara dokumen, menemukan topik abstrak, melakukan <i>clustering</i> dokumen, dan sebagainya. Salah satu penerapannya adalah untuk menyusun buku-buku yang memiliki topik yang mirip berdasarkan isi buku-buku tersebut.  

<img src="https://i.ibb.co/0srwgNv/blei.png" alt="blei image" style="width:75%; height:50%;">
<center>Sumber gambar: Blei, D. M. (2012). Probabilistic topic models. <i>Communications of the ACM, 55</i>(4), 77-84</center>

Terdapat beberapa pendekatan dalam menentukan topik pada suatu teks, misalnya pendekatan Term Frequency and Inverse Document Frequency (TF-IDF), Non-Negative Matrix Factorization (NMF), Latent Semantic Analysis (LSA), dan Latent Dirichlet Allocation (LDA). Pada tutorial kali ini akan dilakukan <i>topic modeling</i> menggunakan pendekatan LDA. Pendekatan LDA dalam <i>topic modeling</i> menganggap setiap dokumen sebagai kumpulan topik dalam proporsi tertentu dan setiap topik sebagai kumpulan kata kunci dalam proporsi tertentu. Topik pada LDA merupakan kumpulan kata kunci dominan yang dapat mewakilkan suatu ciri khas. 

<img src="https://i.ibb.co/QKpbhQx/lda.png" alt="lda image" style="width:75%; height:50%;">
<center>Sumber gambar: http://chdoig.github.io/pytexas2015-topic-modeling/#/3/4</center>


Pada self learning ini, Anda diminta untuk melakukan <i>topic modelling</i> menggunakan pendekatan LDA dengan dataset yang sudah disediakan. Langkah-langkah untuk melakukan <i>topic modelling</i> dengan pendekatan LDA menggunakan <i>dataset</i> tersebut adalah sebagai berikut:

## <b>1. Import library</b><br>
Import seluruh <i>library</i> yang dibutuhkan pada tutorial ini.

<b>#Code 1</b><br>
<i>Code</i> ini digunakan untuk melakukan unduhan dan import library yang diperlukan

In [ ]:
!pip install nltk
!pip install plotly
!pip install numpy
!pip install pandas
!pip install pyLDAvis
!pip install gensim
!pip install scikit-learn
!pip install tqdm

import os
import numpy as np
import nltk
import re
import pandas as pd
import plotly as py
import matplotlib.pyplot as plt
import plotly.graph_objs as go

import pyLDAvis.gensim_models
# jika masih error "Module not found" setelah instalasi pyldavis, 
# ubah import menjadi "pyLDAvis.gensim" dan semua kemunculan pyLDAvis di bawahnya

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from collections import OrderedDict, Counter
from sklearn.decomposition import NMF
from wordcloud import WordCloud
from tqdm import tqdm

from gensim import corpora, models, similarities
from gensim.models import CoherenceModel

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

nltk.download('punkt')

import warnings
warnings.filterwarnings("ignore")

## <b>2. Membaca data</b><br>
Dataset yang digunakan adalah tweet yang diambil langsung dengan metode scraping melalui media sosial twitter dengan 3 keyword besar yang berbeda dengan rentang waktu 8 Februari 2022 hingga 8 Maret 2022. Setiap keyword dilakukan scrape secara terpisah dan kemudian masing - masing hasilnya digabungkan menjadi satu kesatuan dataset.

Keyword yang digunakan pada dataset yang digunakan adalah:
1. "perang rusia ukraina" -> Diterjemahkan sebagai "perang AND rusia AND ukraina"
2. "masker covid" -> Diterjemahkan sebagai "masker AND covid"
3. "makan keluarga" -> Diterjemahkan sebagai "makan AND keluarga"

Hasil final dari data yang digunakan saat ini berupa data bernama dataset.csv yang berisi 41721 baris data. Dataset akan dibaca dengan menggunakan <i>library</i> bernama pandas.

<b>#Code 2a</b><br>
<i>Code</i> ini digunakan untuk mengambil data dari sumber utama serta membaca dataset utama yang berisi tweet yang akan diolah

In [ ]:
# Mengambil data
!wget -q --no-check-certificate 'https://docs.google.com/uc?export=download&id=1rFLB_1QpZhKyWFGxqPo5znTEkh7dUhPf' -O dataset.csv
!wget -q --no-check-certificate 'https://docs.google.com/uc?export=download&id=1xEIQwYre1SR71uRdQuez9MDmAUIysvAG' -O stopwordsID.csv
!wget -q --no-check-certificate 'https://docs.google.com/uc?export=download&id=19NOzXA8Voturopg_DTuiMns3s4M2IAUz' -O kamus_singkatan.csv
!wget -q --no-check-certificate 'https://docs.google.com/uc?export=download&id=1VjgivEr1pxyRCuyhVifPnaReFz0yd8Us' -O colloquial-indonesian-lexicon.csv

# Membaca dataset
datafile = 'dataset.csv'
tweets = pd.read_csv(datafile)
tweets = tweets.drop_duplicates(subset='text')
tweets = tweets.assign(created_at=pd.to_datetime(tweets.created_at))
print('Terdapat ' + str(tweets.shape[0]) + ' Baris data tanpa duplikat tweet yang akan diolah')
tweets.head()

<b>#Code 2b</b><br>
<i>Code</i> ini digunakan untuk membuat grafik terkait aktivitas tweet sepanjang waktu

In [ ]:
# Melihat aktivitas tweet sepanjang waktu
tweets['created_at'] = pd.to_datetime(tweets['created_at'], format='%y-%m-%d %H:%M:%S')
tweetsT = tweets['created_at']

trace = go.Histogram(
    x=tweetsT,
    marker=dict(
        color='blue'
    ),
    opacity=0.75
)

layout = go.Layout(
    title='Aktivitas tweet sepanjang waktu',
    height=450,
    width=1200,
    xaxis=dict(
        title='Bulan dan tahun'
    ),
    yaxis=dict(
        title='Kuantitas tweet'
    ),
    bargap=0.2,
)

data = [trace]

fig = go.Figure(data=data, layout=layout)
py.offline.iplot(fig)

## <b>3. Pra-pemrosesan data</b><br>
Data yang berasal dari Twitter, atau lebih tepatnya adalah data yang berupa teks adalah data bersifat tidak terstruktur dalam artian data tersebut memiliki format penulisan bebas (tidak sesuai kaidah penulisan yang benar, misalnya kaidah penulisan ejaan baku bahasa indonesia). Oleh karena itu, perlu untuk dilakukan pra-pemrosesan data untuk melakukan normalisasi isi <i>tweet</i>. 

Dalam percobaan ini, dilakukan pra-pemrosesan berupa:

    a. Pembersihan tweet
    Adapun yang dilakukan untuik membersihkan tweet: lowercasing; menghapus url, string b' & RT, emoji, url, username, hashtag, tanda baca, angka, spasi berlebih, trimming, pembuangan tanda baca, dan huruf berulang. Fungsi cleaning(tweet) menerima input berupa satu buah tweet mentah bertipe string. 

<b>#Code 3a</b><br>
<i>Code</i> ini digunakan untuk pembersihan dari setiap tweet yang akan diolah, hal ini dimaksudkan agar mendapatkan data tweet yang lebih "bersih" dan lebih mudah untuk dilakukan pengolahan lebih lanjut

In [ ]:
def clean_data(tweet):
    # lowercase
    normal_tw = tweet.lower()
    # hapus b'
    normal_tw = re.sub(r'^b\'', '', normal_tw)
    # hapus RT
    normal_tw = re.sub(r'^rt ', '', normal_tw)
    # hapus emoji
    normal_tw = re.sub(r'\\x.{2}', '', normal_tw)
    # hapus www.* atau https?://* (URL)
    normal_tw = re.sub(r'((www\.[^\s]*)|(https?://[^\s]*))', '', normal_tw)
    # remove spasi berlebih
    normal_tw = re.sub(r'\s+', ' ', normal_tw)
    # trim depan belakang
    normal_tw = normal_tw.strip()
    # regex huruf yang berulang kaya haiiii (untuk fitur unigram)
    normal_regex = re.compile(r"(.)\1{1,}")
    # buang huruf yang berulang
    normal_tw = normal_regex.sub(r"\1\1", normal_tw)
    # hapus @username
    normal_tw = re.sub(r'@[^\s]+', '', normal_tw)
    # hapus hashtag
    normal_tw = re.sub(r'#[^\s]+', '', normal_tw)
    # hapus tanda baca
    normal_tw = re.sub(r'[^\w\s]', '', normal_tw) 
    # hapus angka
    normal_tw = re.sub(r'\d+', ' ', normal_tw) 
    return normal_tw

    b. Pembuangan stopwords dan normalisasi
    Daftar stopwords didapatkan dari penelitian Tala:
    Tala, F. Z. (2003). A Study of Stemming Effects on Information Retrieval in Bahasa Indonesia. M.S. thesis. M.Sc. Thesis. Master of Logic Project. Institute for Logic, Language and Computation. Universiteti van Amsterdam The Netherlands.
    
    Kamus singkatan didapatkan dari penelitian Hakim:
    Hakim, A. N. (2016). Pemrosesan Pertanyaan pada Sistem Tanya Jawab Bidang Kesehatan dengan Pendekatan Pembelajaran Mesin. Bachelor’s Thesis, Universitas Indonesia, Kampus UI Depok.
    
    Kamus alay didapatkan dari penelitian Salsabila:
    N. Aliyah Salsabila, Y. Ardhito Winatmoko, A. Akbar Septiandri and A. Jamal, "Colloquial Indonesian Lexicon," 2018 International Conference on Asian Language Processing (IALP), Bandung, Indonesia, 2018, pp. 226-229, doi: 10.1109/IALP.2018.8629151.

    Fungsi remove_stopwords_and_normalize(tweet) menerima masukan berupa tweet yang telah dibersihkan yang bertipe string.
    
<b># Code 3b</b><br>
<i>Code</i> ini digunakan untuk penghapusan stopwords dan normalisasi untuk setiap tweet yang akan diolah

In [ ]:
def remove_stopwords_and_normalize(tweet):
    token = nltk.word_tokenize(tweet)
    token_new = []
    for k in token:
        if k in df_kamus_singkatan['singkatan'].values:
            k = df_kamus_singkatan.loc[df_kamus_singkatan['singkatan']
                                       == k, 'asli'].values[0]
        if k in df_kamus_alay['slang'].values:
            k = df_kamus_alay.loc[df_kamus_alay['slang']
                                  == k, 'formal'].values[0]
        if k not in stopwords[0].values:
            token_new.append(k)

    str_clean = ' '.join(token_new)
    return str_clean

    c. Pra-pemrosesan tweet secara keseluruhan
    Pada tahap ini, akan dilakukan pemanggilan fungsi clean_data dan remove_stopwords_and_normalize yang sudah didefinisikan sebelumnya. Setelah itu akan ditampilkan sampel tiga tweet pertama hasil pra-pemrosesan.
    
<b>#Code 3c</b><br>
<i>Code</i> ini digunakan untuk memanggil fungsi - fungsi pra pemrosean data yang sebelumnya sudah didefinisikan

In [ ]:
def pra_pemrosesan(list_tweet):
    tweet_clean = []
    for tw in tqdm(list_tweet):
        normal_tweet = clean_data(tw)
        normal_tweet = remove_stopwords_and_normalize(normal_tweet)
        tweet_clean.append(normal_tweet)
    return tweet_clean

In [ ]:
raw_tweet = tweets['text']

stopwords = pd.read_csv("stopwordsID.csv", header=None)
df_kamus_singkatan = pd.read_csv('kamus_singkatan.csv')
df_kamus_alay = pd.read_csv('colloquial-indonesian-lexicon.csv')

# Melakukan pra pemrosesan tweet.
# CATATAN: proses ini akan memakan waktu sekitar 15-20 menit karena banyaknya data yang perlu diproses
print("Sedang memproses tweet... Mohon menunggu")
clean_tweet = pra_pemrosesan(raw_tweet)

# Menampilkan 10 tweet pertama yang telah dibersihkan
clean_tweet[:10]

Jika Anda menemukan bahwa hasil pra pemrosesan belum sempurna hingga menjadi sekumpulan tweet yang mengikuti ejaan bahasa indonesia baku, hal tersebut bukanlah masalah, sebab hingga saat ini memang belum ada cara yang sempurna untuk melakukan pembersihan dan pengolahan awal pada teks bahasa indonesia

## <b>4. Membuat korpus dari seluruh data <i>tweet</i></b><br>
Seluruh teks dalam dokumen <i>tweet</i> digabungkan menjadi korpus.


<b>#Code 4</b><br>
<i>Code</i> ini digunakan untuk membuat korpus yang dibentuk dari semua data tweet yang sudah dilakukan pra pemrosesan

In [ ]:
def create_tweet_corpus(list_tweet):
    corpus = []
    for tweet in list_tweet:
        if not pd.isnull(tweet):
            words = nltk.word_tokenize(tweet)
            for word in words:
                corpus.append([word])
    return corpus

tweet_corpus = create_tweet_corpus(clean_tweet)
tweet_corpus[:10]

## <b> 5. Membuat <i>term dictionary</i></b><br>
Membuat <i>term dictionary</i> dari korpus yang telah dibuat. Setiap <i>term</i> unik diberikan suatu indeks.
    
<b>#Code 5</b><br>
<i>Code</i> ini digunakan untuk membuat dictionary dari korpus, dimana setiap kata unik dipetakan dengan indeks. Pembuatannya sendiri menggunakan library gensim.

In [ ]:
dictionary = corpora.Dictionary(tweet_corpus)

# Mencoba melihat 5 indeks pertama isi dari term dictionary
for index in range(10):
    print(dictionary.get(index))

Bisa dilihat pada output dari kode diatas merupakan hasil pemetaan dari kata - kata yang muncul pada #Code 4 dengan indeks yang dimulai dari 0 hingga N, dimana N adalah banyaknya kata / term. Meskipun begitu, term dictionary dengan library gensim sudah mengolah kata yang menjadi duplikat, sehingga kata yang duplikat tetap berada di indeks yang sama dan tidak dimasukkan dalam indeks yang berbeda.

Contoh:  
Kalimat <b>asyik libur libur libur</b> jika dipetakan menjadi {0: "asyik", 1: "libur"} dan bukan menjadi {0: "asyik", 1: "libur", 2: "libur", 3: "libur"}

## <b> 6. Membuat <i>term document frequency</i></b><br>
Membuat <i>term document frequency</i> dari korpus menggunakan <i>dictionary</i> yang sudah didefinisikan sebelumnya. 

<b>#Code 6</b><br>
<i>Code</i> ini digunakan untuk membuat term document frequency. Berbeda dengan term frequency, pemetaan yang dilakukan adalah pemetaan indeks dengan frekuensi suatu kata pada dokumen.  
  
Contoh:  
Kalimat <b>asyik libur libur libur</b> terdiri dari "asyik" dengan frekuensi 1 kata, dan "libur" dengan frekuensi 3 kata. Dengan demikian term document frequency untuk kalimat tersebut adalah [(0,1),(1,3)] dimana setiap elemen pada array terbut adalah berupa tuple (indeks, frekuensi suatu kata X pada data).

In [ ]:
# Term Document Frequency: converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
corpus = [dictionary.doc2bow(text) for text in tweet_corpus]

# Menampilkan 10 data pertama
print(corpus[:10])

## <b>7. Pembentukan model & <i>training</i> data menggunakan LDA</b><br>
Melakukan <i>training</i> menggunakan <i>corpus term document frequency</i>. Total topik yang dipilih sebanyak 5. Anda bisa bereksperimen dengan mengganti jumlah total topik dalam melakukan <i>training</i> pada model. Kemudian ditampilkan 10 kata yang paling penting pada setiap topik yang terbentuk pada model.

<b>#Code 7</b><br>
<i>Code</i> ini digunakan untuk membangun model LDA serta melakukan training data menggunakan model LDA yang sudah dibangun.

In [ ]:
tfidf = models.TfidfModel(corpus)  # Tahap 1 -- menginisialisasi model
corpus_tfidf = tfidf[corpus]  # Tahap 2 -- menggunakan model untuk mendapatkan corpus vectors

total_topics = 5 # Jumlah topik = 5
lda = models.ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=total_topics)
corpus_lda = lda[corpus]

# Menampilkan 10 kata paling penting pertama di masing-masing topik:
lda.show_topics(total_topics, 10)

Dari setiap topik yang terbentuk, Anda bisa melihat kata-kata apa saja yang banyak muncul dari topik tersebut. Kemudian Anda bisa menggeneralisir topik yang dibicarakan pada klaster tersebut.

## <b>8. Visualisasi</b><br>
Dalam visualisasi ini, Anda bisa melihat persebaran kata pada setiap topik yang sudah dibentuk

<b>#Code 8a</b><br>
<i>Code</i> ini digunakan untuk menampilkan visualisasi dari model LDA dengan beberapa topic

In [ ]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.gensim_models.prepare(lda, corpus, dictionary, mds='tsne')
panel

<b>#Code 8b</b><br>
<i>Code</i> ini digunakan untuk membuat wordcloud. Word cloud secara umum berisi kata - kata yang muncul dalam data, yang dipetakan berdasarkan frekuensi. Semakin besar ukuran huruf suatu kata pada wordcloud, maka kata tersebut semakin sering muncul pada kumpulan data. Berikut ini adalah wordcloud yang dihasilkan dari semua tweet yang sudah dilakukan preprocessing

In [ ]:
long_string = ';'.join(clean_tweet)
wordcloud = WordCloud(background_color='white', max_words=5000, 
                      contour_width=3, contour_color='steelblue')
wordcloud.generate(long_string)
wordcloud.to_image()

## <b>9. Evaluasi</b><br>
Evaluasi merupakan langkah yang penting agar dapat mengetahui bagaimana kualitas dari topic modelling yang telah dibangun. Pada bagian ini akan dilakukan evaluasi model yang dibentuk menggunakan LDA dengan meghitung nilai koherensi nya (Coherence) dimana semakin tinggi nilainya maka semakin baik juga hasil topic modelling yang telah dibangun.  
Evaluasi menggunakan dua jenis skor, Perplexity dan Coherence. 

<b>#Code 9</b><br>
<i>Code</i> ini digunakan untuk melakukan evaluasi LDA dengan membentuk fungsi - fungsi yang bersifat reusable

In [ ]:
def calculate_perplexity(corpus):
    return lda.log_perplexity(corpus)

def calculate_coherence(model, word_corpus, dictionary):
    coherence_model_lda = CoherenceModel(model=model, texts=word_corpus, dictionary=dictionary, coherence='c_v')
    return coherence_model_lda.get_coherence()

In [ ]:
# Menghitung nilai perplexity untuk mengetahui seberapa baik model yang dibangun, semakin kecil nilainya maka semakin baik kualitasnya
print('\nNilai Perplexity: ', calculate_perplexity(corpus))

# Menghitung nilai koherensi / coherence. Rentang nilai berada di 1 < x < 0. Nilai yang bagus biasanya berada di rentang 0,85 < x < 0,65
print('\nNilai Coherence: ', calculate_coherence(lda, tweet_corpus, dictionary))

Apakah nilai tersebut sudah mencerminkan bahwa hasil topic modelling yang dirancang sudah baik ? Belum tentu, karena untuk mendapatkan hasil yang lebih baik, salah satu faktor yang dapat diperbaiki adalah menentukan jumlah topic yang akan dipetakan. Saat ini hanya menggunakan 5 topic (k=5), namun nilai topic bisa berbeda. Lantas bagaimana menentukan nilai yang lebih baik ? Akan dijelaskan dibawah ini

### <b>Mencari jumlah topic yang optimal untuk LDA</b>
Pendekatan yang digunakan disini adalah melakukan iterasi nilai K (nilai besarnya angka topic) dari rentang inklusif [x,y] kemudian untuk setiap nilai K, hitung nilai perplexity dan coherencenya.

<b>#Code 9.1a</b><br>
<i>Code</i> ini berisi fungsi yang akan digunakan untuk mencari nilai topic terbaik dan melakukan visualisasi

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start, step):
    """
    Fungsi menghitung coherence (c_v) untuk berbagai nilai topic.

    Parameter:
    ----------
    dictionary : Gensim dictionary
    corpus : Korpus data asli
    texts : Korpus tweet yang berada di #Code 4
    start : nilai awal topic
    limit : batas nilai topic
    step : increment nilai topic

    Pada contoh sebelumnya, nilai topic adalah 5, namun sekarang nilai topic
    di-iterasi dari rentang [start, limit] dengan incremental sebesar 'step'

    Keluaran:
    -------
    model_list : List dari LDA topic modelling yang bersesuaian dengan nilai topic tertentu
    coherence_values : Nilai coherence dari LDA topic modelling yang bersesuaian dengan nilai topic tertentu
    """
    model_list = []
    coherence_values = []
    perplexity_values = []
    for num_topics in range(start, limit, step):
        print("Saat ini mengolah nilai topic = ", num_topics)
        # Menambahkan model - model LDA
        model = models.ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=num_topics)
        model_list.append(model)
        # Menambahkan nilai - nilai coherence
        coherencemodel = calculate_coherence(model, texts, dictionary)
        coherence_values.append(coherencemodel)
        # Menambahkan nilai - nilai perplexity
        perplexityvalue = calculate_perplexity(corpus)
        perplexity_values.append(perplexityvalue)
    return model_list, coherence_values, perplexity_values

def visualize_perplexity(perplexity_values, start, limit, step):
    x = range(start, limit, step)
    plt.plot(x, perplexity_values)
    plt.xlabel("Besar nilai perplexity")
    plt.ylabel("Nilai perplexity")
    plt.legend(("perplexity_values"), loc='best')
    plt.show()

def visualize_coherence(coherence_values, start, limit, step):
    x = range(start, limit, step)
    plt.plot(x, coherence_values)
    plt.xlabel("Besar nilai topic")
    plt.ylabel("Nilai coherence")
    plt.legend(("coherence_values"), loc='best')
    plt.show()

<b>#Code 9.1b</b><br>
<i>Code</i> ini berfungsi untuk menjalankan pemanggilan fungsi untuk mendapatkan daftar model, daftar nilai coherence, dan daftar nilai perplexity

In [ ]:
# PERHATIAN: Proses ini bisa memakan waktu 10-30 menit
TOPIC_START = 1
TOPIC_LIMIT = 13
TOPIC_STEP = 5

# Pembentukan daftar nilai model dan nilai coherence nya
model_list, coherence_values, perplexity_values = compute_coherence_values(
    dictionary=dictionary,
    corpus=corpus,
    texts=tweet_corpus,
    start=TOPIC_START,
    limit=TOPIC_LIMIT,
    step=TOPIC_STEP
)

# Visualisasi nilai perplexity
visualize_perplexity(perplexity_values, TOPIC_START, TOPIC_LIMIT, TOPIC_STEP)

# Visualisasi nilai topic
visualize_coherence(coherence_values, TOPIC_START, TOPIC_LIMIT, TOPIC_STEP)

## <b>10. Topic Modelling dengan Metode Lain</b><br>
Pada self pace learning kali ini akan menggunakan K-Means Clustering dan NNMF (Non Negative Matrix Factorization)

### <b>1. Pemodelan topik menggunakan K-Means Clustering</b><br>
Salah satu algoritma <i>clustering</i> lainnya yang umum digunakan adalah K-Means. Berikut adalah kode untuk mengeksekusi algoritma K-Means menggunakan library `sklearn`. Anda dapat membandingkan hasil <i>clustering</i> yang dilakukan oleh LDA dengan K-Means.

<b>#Code 10.1a</b><br>
<i>Code</i> ini digunakan untuk membangun model K-Means Clustering dengan nilai K=5

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(clean_tweet)

# Train algoritma K-Means
kmeans = KMeans(n_clusters=5) # jumlah klaster = 5
kmeans.fit(X)

<b>#Code 10.1b</b><br>
<i>Code</i> ini digunakan menampilkan beberapa hasil kata dari cluster yang sudah dibangun

In [ ]:
# Membuat dictionary untuk data cluster
clusters = {}

for tweet, cluster in zip(clean_tweet, kmeans.labels_.tolist()):
    if cluster not in clusters:
        clusters[cluster] = [tweet]
    else:
        clusters[cluster].append(tweet)

# Menampilkan sejumlah n=20 kata-kata dominan di masing-masing cluster
n_words = 20
for cluster, tweets in clusters.items():
    long_string = " ".join(tweets)
    words = nltk.word_tokenize(long_string)
    dominant_words = pd.Series(words).value_counts()[:n_words].index.to_list()

    print("Cluster " + str(cluster))
    print(" | ".join(dominant_words))
    print('='*90)

### <b>2. Pemodelan topik menggunakan NNMF</b><br>

NNMF atau yang biasa disebut dengan Non-Negative Matrix Factorization sejatinya adalah metode statistik yang digunakan untuk menurunkan dimensi dari input yang berupa korpus. NNMF menggunakan metode analisis faktor untuk memberikan pemetaan kata - kata yang memiliki koherensi rendah dengan kata yang berbobot rendah. Cara kerja NNMF adalah dengan melakukan <i>decompose</i> vektor berdimensi tinggi men jadi vektor berdimensi rendah. Dengan menggunakan sebuah matriks (A), NNMF akan menghasilkan dua buah matriks (B dan C) dimana matriks X berisi topic yang ditemukan, dan matriks Y berisi bobot dari topik - topik tersebut.

Asumsikan ada 400  (Baris data), 5000 kata (Total) dan 50 topik, maka akan menghasilkan matriks berikut:
- A = 400 x 5000
- B = 5000 x 50
- C = 50 x 400

<b>#Code 10.2a</b><br>
<i>Code</i> ini digunakan untuk melakukan transformasi input data

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(clean_tweet)
words = np.array(vectorizer.get_feature_names())

<b>#Code 10.2b</b><br>
<i>Code</i> ini digunakan menampilkan beberapa topic yang sudah dibangun

In [ ]:
nmf = NMF(n_components=5, solver="mu")
B = nmf.fit_transform(X)
C = nmf.components_

for i, topic in enumerate(C):
     print("Topic {}: {}".format(i + 1, ",".join([str(x) for x in words[topic.argsort()[-10:]]])))

print("")
print("Matriks B ada sebanyak " + str(len(B)) + " baris dan " + str(len(B[0])) + " kolom")
print("Matriks C ada sebanyak " + str(len(C)) + " baris dan " + str(len(C[0])) + " kolom")

# <b><font color='red'>B. Tugas Self Paced Learning - Topic Modelling</font></b>

1. <b>[10 Poin]</b> Terlihat pada hasil keluaran kode pada cell #Code2a tanggal 24 Februari 2022 dan 4 Maret 2022 terjadi lonjakan aktivitas yang cukup tajam. Menurut Anda berdasarkan ketiga keyword utama yang berperan membangun dataset ini (Refer ke Section 2), keyword manakah yang menyebabkan adanya lonjakan aktivitas pada tanggal tersebut (Keyword 1/2/3) ? dan berikan alasannya.

2. <b>[10 Poin]</b> Menurut Anda, apa tujuan utama dari <i>pembentukan term document dan term document frequency</i> ? Apakah ada keuntungan dan kerugian yang mungkin diakibatkan dari proses ini apabila teknik ini tidak hanya diterapkan pada topic modelling, tapi pada aktivitas lain yang membutuhkan manipulasi data tekstual ?

3. <b>[10 Poin]</b> Berdasarkan hasil dari <b>#Code 7</b> apa maksud dari angka desimal yang muncul disamping setiap kata pada hasil keluaran kode? Berikan penjelasan Anda

4. <b>[30 Poin]</b> Cobalah untuk menjalankan tahap <b>4. Membuat korpus dari seluruh data tweet</b> hingga akhir tetapi menggunakan data tweet indeks inklusif [0,10000] , [20000,30000], dan [30000,40000] dalam percobaan terpisah, dengan begitu akan ada 3x percobaan). Apakah masing - masing penggolongan topic yang dihasilkan lebih baik (berdasarkan pengetahuan manusia) daripada apa yang dihasilkan di <b>#Code 8a</b> ? Jelaskan jawaban Anda

Catatan nomor 4, perbandingan yang dimaksud adalah
- Data indeks [10,20000] vs hasil dari <b>#Code 8a</b>
- Data indeks [20000,30000] vs hasil dari <b>#Code 8a</b>
- Data indeks [30000,akhir] vs hasil dari<b>#Code 8a</b>

5. <b>[40 Poin]</b> Anda diberikan dataset baru dengan keyword tertentu yang diambil dari tanggal 1 Januari 2021 hingga 31 Desember 2021 dengan maksimal 11000 tweets. Silakan lakukan semua tahap yang sudah diajarkan pada self paced learning ini dari mulai tahap preprocessing hingga pembentukan visualisasi dari topic modelling (awal hingga akhir). Jelaskan setiap tahap yang dilakukan dan jelaskan juga apa hasil serta kesimpulan atau hal menarik apa yang Anda dapatkan!

Ketentuan dataset:
Dataset sudah disediakan di cell bawah, cukup gunakan kolom `id`, `text` dan `created_at` saja
- Laki - laki dengan angka terakhir NPM GANJIL: keyword `menguasai dunia`
- Laki - laki dengan angka terakhir NPM GENAP: keyword `butuh liburan`
- Perempuan dengan angka terakhir NPM GANJIL: keyword `makan seblak`
- Perempuan dengan angka terakhir NPM GENAP: keyword `nonton kpop`

Jawaban nomor 5 harus mengandung:
- Preprosessing
- Pembuatan korpus
- Membuat term dictionary & term document frequency
- Pembentukan model dengan LDA
- Visualisasi
- Evaluasi

Selamat Mengerjakan!  
Jangan lupa berdoa, makan dan mencuci tangan!

In [ ]:
# Dataset keyword "menguasai dunia", silakan di-uncomment semua baris di bawah ini

# !wget -q --no-check-certificate 'https://docs.google.com/uc?export=download&id=1p2BMF7GDZA6u3wmvbxMO5bs3QxU4m8A_' -O menguasai_dunia.csv
# datafile_a = 'menguasai_dunia.csv'
# tweets_a = pd.read_csv(datafile_a)
# tweets_a

In [ ]:
# Dataset keyword "butuh liburan", silakan di-uncomment semua baris di bawah ini

# !wget -q --no-check-certificate 'https://docs.google.com/uc?export=download&id=1aGY6y9vABCKD08MkA6Rwv2FkALHf5VbY' -O butuh_liburan.csv
# datafile_b = 'butuh_liburan.csv'
# tweets_b = pd.read_csv(datafile_b)
# tweets_b

In [ ]:
# Dataset keyword "makan seblak", silakan di-uncomment semua baris di bawah ini

# !wget -q --no-check-certificate 'https://docs.google.com/uc?export=download&id=1x-8ev32dumbu0zENIrEtIJQPfmqKk6H-' -O makan_seblak.csv
# datafile_c = 'makan_seblak.csv'
# tweets_c = pd.read_csv(datafile_c)
# tweets_c

In [ ]:
# Dataset keyword "nonton kpop", silakan di-uncomment semua baris di bawah ini

# !wget -q --no-check-certificate 'https://docs.google.com/uc?export=download&id=1IzTzONy2Ih5YyXQoMYDd60Nh79YakmQ4' -O nonton_kpop.csv
# datafile_d = 'nonton_kpop.csv'
# tweets_d = pd.read_csv(datafile_d)
# tweets_d